In [2]:
import os
import sys
import typing as tp

import joblib
import numpy as np
import pandas as pd
import tqdm.notebook
from recsys.datasets import ml1m, ml100k
from sklearn.preprocessing import LabelEncoder

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
ratings, movies = ml100k.load()
ratings.head()

,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [5]:
movies.head()

,itemid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
my_favourite_movies = [1, 96, 100, 23, 29, 50, 67, 72, 89, 95, 98, 121, 127, 135, 151, 161, 178, 181, 196, 204, 210, 225, 222, 227, 228, 229, 230, 231, 250]

my_favourite_movies5 = [1, 29, 50, 67, 95, 98, 121, 127, 178, 181, 225, 222, 227, 231, 250]

my_favourite_movies4 = [96, 100, 23, 72, 89, 135, 151, 161, 196, 204, 210, 228, 229, 231]
assert len(my_favourite_movies4) + len(my_favourite_movies5) == len(my_favourite_movies)

my_user_index = max(ratings['userid']) + 1
for id in my_favourite_movies5:
    new_rows = pd.DataFrame({
        'userid': [my_user_index],
        'itemid': [id],
        'rating': [5]
    }, index=[0])
    ratings = pd.concat([ratings, new_rows], ignore_index=True)
for id in my_favourite_movies4:
    new_rows = pd.DataFrame({
        'userid': [my_user_index],
        'itemid': [id],
        'rating': [4]
    }, index=[0])
    ratings = pd.concat([ratings, new_rows], ignore_index=True)
ratings

,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
100024,944,204,4
100025,944,210,4
100026,944,228,4
100027,944,229,4


### Preprocessing

In [7]:
def ids_encoder(ratings):
    users = sorted(ratings["userid"].unique())
    items = sorted(ratings["itemid"].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings.userid = uencoder.transform(ratings.userid.tolist())
    ratings.itemid = iencoder.transform(ratings.itemid.tolist())

    return ratings, uencoder, iencoder

In [8]:
# create the encoder
ratings, uencoder, iencoder = ids_encoder(ratings)
ratings

,userid,itemid,rating
0,0,0,5
1,0,1,3
2,0,2,4
3,0,3,3
4,0,4,3
...,...,...,...
100024,943,203,4
100025,943,209,4
100026,943,227,4
100027,943,228,4


In [24]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ratings, test_size=0.2)
train_data, valid_data = train_test_split(train_data, test_size=0.25)

np_ratings = train_data.to_numpy()
all_ratings = ratings.to_numpy()

## 1 Задание

In [178]:
from functools import lru_cache

@lru_cache(5000)
def ratings_for_user(i):
    return all_ratings[all_ratings[:, 0] == i]

### Коэффициент Жаккара

In [179]:
def jaccard_similarity(np_ratings, i: int, j: int) -> float:
    if i == j:
        return 1.0

    ratings_i, ratings_j = ratings_for_user(i), ratings_for_user(j)
    movies_user1 = ratings_i[:, 1]
    movies_user2 = ratings_j[:, 1]

    common_movies = np.intersect1d(movies_user1, movies_user2)
    all_movies = np.union1d(movies_user1, movies_user2)

    jaccard_index = len(common_movies) / len(all_movies)

    return jaccard_index

assert np.isclose(jaccard_similarity(np_ratings, 0, 0), 1.0)
print(jaccard_similarity(np_ratings, 0, 1))

0.056962025316455694


### Скалярное произведение общих рейтингов

In [188]:
def dot_product_similarity(np_ratings, i: int, j: int) -> float:
    if i == j:
        return 1.0

    ratings_i, ratings_j = ratings_for_user(i), ratings_for_user(j)

    common_movies = np.intersect1d(ratings_i[:, 1], ratings_j[:, 1])

    common_ratings_i = ratings_i[np.isin(ratings_i[:, 1], common_movies)]
    common_ratings_j = ratings_j[np.isin(ratings_j[:, 1], common_movies)]

    x = common_ratings_i[:, 2]
    y = common_ratings_j[:, 2]

    similarity = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
    if not np.isnan(similarity):
        return similarity
    else:
        return 0

print(dot_product_similarity(np_ratings, 0, 1))
assert np.isclose(dot_product_similarity(np_ratings, 0, 0), 1.0)

0.9605819645600208


### Скорректированная Кореляция Пирсона

In [181]:
def pearson_correlation(np_ratings, i: int, j: int) -> float:
    if i == j:
        return 1.0

    ratings_i, ratings_j = ratings_for_user(i), ratings_for_user(j)

    mean_user1 = np.mean(ratings_i[:, 2])
    mean_user2 = np.mean(ratings_j[:, 2])

    numerator = sum([(rating_user1 - mean_user1) * (rating_user2 - mean_user2) for rating_user1, rating_user2 in zip(ratings_i[:, 2], ratings_j[:, 2])])
    denominator = np.sqrt(sum([(rating_user1 - mean_user1)**2 for rating_user1 in ratings_i[:, 2]]) * sum([(rating_user2 - mean_user2)**2 for rating_user2 in ratings_j[:, 2]]))

    pearson_corr = numerator / denominator if denominator != 0 else numerator

    common_movies = np.intersect1d(ratings_i[:, 1], ratings_j[:, 1])
    coef = min(1, len(common_movies) / 50)
    if np.isclose(coef, 0.0):
        coef = 1
    return pearson_corr * coef

print(pearson_correlation(np_ratings, 0, 1))
print(pearson_correlation(np_ratings, 2, 3))
assert np.isclose(pearson_correlation(np_ratings, 0, 0), 1.0)

-0.0017341664853389787
-0.055524917387262014


### 2 Задание

### Простое усреднение по ближайшим соседям

In [14]:
def get_neighbors(np_ratings, user, movie_id, similarity_func, k=50):
    users_similarity = []
    users = np.unique(np_ratings[np_ratings[:, 1] == movie_id][:, 0])
    for cur_user in users:
        if cur_user != user:
            similarity = similarity_func(np_ratings, cur_user, user)
            users_similarity.append((cur_user, similarity))

    users_similarity = sorted(users_similarity, key=lambda x: x[1], reverse=True)
    k = min(k, len(users_similarity))
    top_k_neighbors = users_similarity[:k]

    return top_k_neighbors

get_neighbors(np_ratings, 0, 0, pearson_correlation)

[(215, 0.19198244073602577),
 (289, 0.15562120535833365),
 (520, 0.1541336664798384),
 (902, 0.14563465544661258),
 (880, 0.13499841828857773),
 (393, 0.1300649526313339),
 (746, 0.12762826201639674),
 (338, 0.11971270022230987),
 (756, 0.11654563916696431),
 (304, 0.11219384099355088),
 (842, 0.11190864773468465),
 (291, 0.11101521774217658),
 (5, 0.10884336960670629),
 (822, 0.10425880712130255),
 (177, 0.09892540222191905),
 (882, 0.09855213929887945),
 (886, 0.0962804110066975),
 (649, 0.09481845681661784),
 (57, 0.0933495883242139),
 (587, 0.09077879151430997),
 (652, 0.08831161799534844),
 (535, 0.08639127789745854),
 (477, 0.08456780665622526),
 (891, 0.08196022132409109),
 (683, 0.08150522168254352),
 (349, 0.0765146463935229),
 (664, 0.07640379163899759),
 (544, 0.07526396656320619),
 (804, 0.07269969765656498),
 (793, 0.07247636462903895),
 (592, 0.07088775761687334),
 (647, 0.07030212871394699),
 (342, 0.06901479361953161),
 (326, 0.06825960537811308),
 (660, 0.0681421039435

In [15]:
np_ratings

array([[ 451,  244,    2],
       [ 654, 1637,    3],
       [ 866,  167,    4],
       ...,
       [ 773,  417,    2],
       [ 415,  497,    4],
       [ 267,  204,    5]])

In [16]:
from functools import lru_cache

@lru_cache(5000)
def movie_rating_for_user(user, movie_id) -> float:
    ratings = ratings_for_user(user)
    return ratings[ratings[:, 1] == movie_id][0][2]

assert movie_rating_for_user(917, 198) == 3

In [17]:
def rating_predictions_1(np_ratings, user, movie_id, similarity_func, k=50):
   neighbors = get_neighbors(np_ratings, user, movie_id, similarity_func, k)

   numerator = sum([similarity * movie_rating_for_user(user1, movie_id) for user1, similarity in neighbors])
   denominator = sum([abs(similarity) for user1, similarity in neighbors])

   return numerator / denominator

rating_predictions_1(np_ratings, 0, 0, pearson_correlation)

4.064333352703202

### Усреднение с учётом коррекции среднего

In [18]:
def rating_predictions_2(np_ratings, user, movie_id, similarity_func, k=50):
   neighbors = get_neighbors(np_ratings, user, movie_id, similarity_func, k)

   ratings = ratings_for_user(user)
   mean_user = np.mean(ratings[:, 2])

   numerator = sum([similarity * (movie_rating_for_user(user1, movie_id) - np.mean(ratings_for_user(user1)[:, 2])) for user1, similarity in neighbors])
   denominator = sum([abs(similarity) for user1, similarity in neighbors])

   if denominator == 0.0:
       return mean_user

   return mean_user + (numerator / denominator)

rating_predictions_2(np_ratings, 0, 0, pearson_correlation)

4.057041006395044

### 3 Задание

### Разделите датасет movielens на тренировочную и валидационную части. Постройте рекомендации для пользователей из валидационной части

In [19]:
test_data.head()

,userid,itemid,rating
21216,215,200,3
7448,69,337,2
95886,896,973,4
65477,591,143,5
49644,441,228,3


In [25]:
valid_data.head()

,userid,itemid,rating
346,2,301,2
96076,899,293,4
66021,594,819,2
90003,848,171,5
35535,326,734,2


In [20]:
def recommend_movies(np_ratings, user_id, item_id):
    return rating_predictions_2(np_ratings, user_id, item_id, pearson_correlation)

In [27]:
valid_data['rec_rating'] = valid_data.apply(lambda row: recommend_movies(np_ratings, row['userid'], row['itemid']), axis=1)
valid_data['rating'] = valid_data.apply(lambda row: float(row['rating']), axis=1)

valid_data.head()

,userid,itemid,rating,rec_rating
346,2,301,2.0,3.459586
96076,899,293,4.0,1.939842
66021,594,819,2.0,3.123016
90003,848,171,5.0,5.519417
35535,326,734,2.0,3.730626


### Метрики для предсказаний

In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error


rmse = np.sqrt(mean_squared_error(valid_data['rating'], valid_data['rec_rating']))
mae = mean_absolute_error(valid_data['rating'], valid_data['rec_rating'])

valid_data['predicted_class'] = valid_data['rec_rating'].round(0).astype(int)

precision = precision_score(valid_data['rating'], valid_data['predicted_class'], average = 'weighted')
recall = recall_score(valid_data['rating'], valid_data['predicted_class'], average = 'weighted')
f1 = f1_score(valid_data['rating'], valid_data['predicted_class'], average = 'weighted')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
valid_data

Precision: 0.44
Recall: 0.41
F1-score: 0.37
RMSE: 0.9840322929646342
MAE: 0.7707491629363995


/Users/tyapkinpetr/PycharmProjects/recsys_v1/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,userid,itemid,rating,rec_rating,predicted_class
346,2,301,2.0,3.459586,3
96076,899,293,4.0,1.939842,2
66021,594,819,2.0,3.123016,3
90003,848,171,5.0,5.519417,6
35535,326,734,2.0,3.730626,4
...,...,...,...,...,...
79390,732,457,2.0,2.855937,3
29537,289,929,3.0,3.304163,3
68810,626,561,2.0,3.255726,3
47940,426,679,5.0,4.235880,4


### 4 Задание

### Выберите от 10 до 50 своих любимых фильмов

In [29]:
ratings, movies = ml100k.load()
my_user_index
ratings

,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
99995,943,1067,2
99996,943,1074,4
99997,943,1188,3
99998,943,1228,3


In [45]:
my_favourite_movies = [0, 95, 99, 22, 28, 49, 66, 71, 88, 94, 97, 120, 126, 134, 150, 160, 177, 180, 195, 203, 209, 224, 221, 226, 227, 228, 229, 230, 249]

In [34]:
np_movies = movies.to_numpy()
np_movies[my_favourite_movies]

array([[1, 'Toy Story (1995)'],
       [2, 'GoldenEye (1995)'],
       [3, 'Four Rooms (1995)'],
       ...,
       [1680, 'Sliding Doors (1998)'],
       [1681, 'You So Crazy (1994)'],
       [1682, 'Scream of Stone (Schrei aus Stein) (1991)']], dtype=object)

In [175]:
my_favourite_movies5 = [0, 28, 49, 66, 94, 97, 120, 126, 177, 180, 224, 221, 226, 230, 249]

my_favourite_movies4 = [95, 99, 22, 71, 88, 134, 150, 160, 195, 203, 209, 227, 228, 229]

assert len(np.intersect1d(my_favourite_movies5, my_favourite_movies4)) == 0
assert len(my_favourite_movies4) + len(my_favourite_movies5) == len(my_favourite_movies)

my_user_index = max(ratings['userid']) + 1
for id in my_favourite_movies5:
    new_rows = pd.DataFrame({
        'userid': [my_user_index],
        'itemid': [id],
        'rating': [5]
    }, index=[0])
    ratings = pd.concat([ratings, new_rows], ignore_index=True)
for id in my_favourite_movies4:
    new_rows = pd.DataFrame({
        'userid': [my_user_index],
        'itemid': [id],
        'rating': [4]
    }, index=[0])
    ratings = pd.concat([ratings, new_rows], ignore_index=True)

In [177]:
ratings, uencoder, iencoder = ids_encoder(ratings)
np_ratings = ratings.to_numpy()
all_ratings = ratings.to_numpy()
np_ratings

array([[  0,   0,   5],
       [  0,   1,   3],
       [  0,   2,   4],
       ...,
       [952, 227,   4],
       [952, 228,   4],
       [952, 229,   4]])

### Топ-10 рекомендаций по каждому из 6 методов

In [166]:
def candidate_items(
    np_ratings: np.array, userid: int, k=-1
) -> tp.Tuple[np.array, np.array]:
    user_movies = np_ratings[np_ratings[:, 0] == userid][:, 1]
    all_movies = np.unique(np_ratings[:, 1])

    return np.setdiff1d(all_movies, user_movies)

In [167]:
candidates = candidate_items(np_ratings, my_user_index - 1)

print("Candidates:", len(candidates))
candidates

Candidates: 1654


array([   1,    2,    3, ..., 1679, 1680, 1681])

In [168]:
def topn_recommendations(rating_predictions, similarity_func, k = 10):
    candidates_with_ratings = []
    for id in candidates:
        candidates_with_ratings.append((rating_predictions(np_ratings, my_user_index - 1, id, similarity_func), id))
    candidates_with_ratings = sorted(candidates_with_ratings, reverse=True)
    return candidates_with_ratings[:k]

In [169]:
ratings_i = ratings_for_user(my_user_index - 1)
movies_user1 = ratings_i[:, 1]
movies_user1

array([], dtype=int64)

In [172]:
top1 = topn_recommendations(rating_predictions_1, jaccard_similarity)
np_movies[[item[1] for item in top1]]

/var/folders/51/m95j7hvn1rg7qdby5h40q6brm6j4p4/T/ipykernel_26550/4148256719.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return numerator / denominator


array([[1201, 'Marlene Dietrich: Shadow and Light (1996) '],
       [1122, 'They Made Me a Criminal (1939)'],
       [814, 'Great Day in Harlem, A (1994)'],
       [1189, 'Prefontaine (1997)'],
       [119, 'Maya Lin: A Strong Clear Vision (1994)'],
       [1233, 'Nénette et Boni (1996)'],
       [408, 'Close Shave, A (1995)'],
       [64, 'Shawshank Redemption, The (1994)'],
       [169, 'Wrong Trousers, The (1993)'],
       [174, 'Raiders of the Lost Ark (1981)']], dtype=object)

In [189]:
top2 = topn_recommendations(rating_predictions_1, dot_product_similarity)
print(top2)
np_movies[[item[1] for item in top2]]

/var/folders/51/m95j7hvn1rg7qdby5h40q6brm6j4p4/T/ipykernel_26550/419510965.py:15: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
/var/folders/51/m95j7hvn1rg7qdby5h40q6brm6j4p4/T/ipykernel_26550/4148256719.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return numerator / denominator


[(5.0, 1200), (5.0, 1188), (5.0, 1121), (5.0, 813), (4.659836305598883, 317), (4.619444146308779, 63), (4.6000648847857155, 11), (4.580134402956076, 482), (4.52013560863347, 407), (4.519492943490043, 602)]


array([[1201, 'Marlene Dietrich: Shadow and Light (1996) '],
       [1189, 'Prefontaine (1997)'],
       [1122, 'They Made Me a Criminal (1939)'],
       [814, 'Great Day in Harlem, A (1994)'],
       [318, "Schindler's List (1993)"],
       [64, 'Shawshank Redemption, The (1994)'],
       [12, 'Usual Suspects, The (1995)'],
       [483, 'Casablanca (1942)'],
       [408, 'Close Shave, A (1995)'],
       [603, 'Rear Window (1954)']], dtype=object)

In [182]:
top3 = topn_recommendations(rating_predictions_1, pearson_correlation)
print(top3)
np_movies[[item[1] for item in top3]]

[(5.0, 1535), (5.0, 1499), (5.0, 1466), (5.0, 1200), (5.0, 1121), (5.0, 813), (4.5635227064721375, 173), (4.561062017052539, 1357), (4.560262806930402, 63), (4.549222301327, 407)]


array([[1536, 'Aiqing wansui (1994)'],
       [1500, 'Santa with Muscles (1996)'],
       [1467, 'Saint of Fort Washington, The (1993)'],
       [1201, 'Marlene Dietrich: Shadow and Light (1996) '],
       [1122, 'They Made Me a Criminal (1939)'],
       [814, 'Great Day in Harlem, A (1994)'],
       [174, 'Raiders of the Lost Ark (1981)'],
       [1358, 'The Deadly Cure (1996)'],
       [64, 'Shawshank Redemption, The (1994)'],
       [408, 'Close Shave, A (1995)']], dtype=object)

In [183]:
top4 = topn_recommendations(rating_predictions_2, jaccard_similarity)
np_movies[[item[1] for item in top4]]

array([[814, 'Great Day in Harlem, A (1994)'],
       [1463, 'Boys, Les (1997)'],
       [1536, 'Aiqing wansui (1994)'],
       [1467, 'Saint of Fort Washington, The (1993)'],
       [1233, 'Nénette et Boni (1996)'],
       [1599, "Someone Else's America (1995)"],
       [1500, 'Santa with Muscles (1996)'],
       [1367, 'Faust (1994)'],
       [1629, 'Nico Icon (1995)'],
       [1642, "Some Mother's Son (1996)"]], dtype=object)

In [190]:
top5 = topn_recommendations(rating_predictions_2, dot_product_similarity)
np_movies[[item[1] for item in top5]]

/var/folders/51/m95j7hvn1rg7qdby5h40q6brm6j4p4/T/ipykernel_26550/419510965.py:15: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


array([[814, 'Great Day in Harlem, A (1994)'],
       [1463, 'Boys, Les (1997)'],
       [1536, 'Aiqing wansui (1994)'],
       [1467, 'Saint of Fort Washington, The (1993)'],
       [1500, 'Santa with Muscles (1996)'],
       [1599, "Someone Else's America (1995)"],
       [1642, "Some Mother's Son (1996)"],
       [1662, 'Rough Magic (1995)'],
       [1233, 'Nénette et Boni (1996)'],
       [1449, 'Pather Panchali (1955)']], dtype=object)

In [119]:
top6 = topn_recommendations(rating_predictions_2, pearson_correlation)
np_movies[[item[1] for item in top6]]

array([[1661, 'New Age, The (1994)'],
       [1626, 'Nobody Loves Me (Keiner liebt mich) (1994)'],
       [1671, 'Further Gesture, A (1996)'],
       [1678, "Mat' i syn (1997)"],
       [1554, 'Safe Passage (1994)'],
       [814, 'Great Day in Harlem, A (1994)'],
       [1256, 'Designated Mourner, The (1997)'],
       [1644, 'Sudden Manhattan (1996)'],
       [1502, 'Naked in New York (1994)'],
       [1257, 'Designated Mourner, The (1997)']], dtype=object)

### Оцените каждый рекомендованный вам фильм по шкале релевантности от 0 до 1. Посчитайте среднюю релевантность и ndcg для каждого метода.\ Релевантны ли вам рекомендации, построенные по вашим фильмам?

In [193]:
relevant_top1 = np.array([0.4, 0.55, 0.7, 0.64, 0.33, 0.4, 0.88, 1, 0.8, 1])
relevant_top2 = np.array([0.4, 0.64, 0.7, 0.7, 1, 1, 1, 0.9, 0.88, 0.9])
relevant_top3 = np.array([0.6, 0.2, 0.35, 0.4, 0.7, 0.7, 1, 0.8, 1, 0.88])
relevant_top4 = np.array([0.7, 0.5, 0.6, 0.35, 0.4, 0.56, 0.2, 0.67, 0.58, 0.48])
relevant_top5 = np.array([0.7, 0.5, 0.6, 0.35, 0.2, 0.56, 0.48, 0.8, 0.4, 0.69])
relevant_top6 = np.array([0.52, 0.45, 0.33, 0.9, 0.6, 0.7, 0.5, 0.4, 0.8, 0.5])

In [196]:
def dcg(relevances):
    n = 10
    log_positions = np.log2(np.arange(n) + 2)
    return np.sum(relevances / log_positions)

def ndcg(recs, k=10):
    dcg_val = dcg(recs)
    idcg_val = dcg(sorted(recs, reverse=True))
    return dcg_val / idcg_val

ndcg_method1 = ndcg(relevant_top1)
ndcg_method2 = ndcg(relevant_top2)
ndcg_method3 = ndcg(relevant_top3)
ndcg_method4 = ndcg(relevant_top4)
ndcg_method5 = ndcg(relevant_top5)
ndcg_method6 = ndcg(relevant_top6)

print("NDCG для метода 1:", ndcg_method1)
print("NDCG для метода 2:", ndcg_method2)
print("NDCG для метода 3:", ndcg_method3)
print("NDCG для метода 4:", ndcg_method4)
print("NDCG для метода 5:", ndcg_method5)
print("NDCG для метода 6:", ndcg_method6)

NDCG для метода 1: 0.8086711892984718
NDCG для метода 2: 0.8473176579515174
NDCG для метода 3: 0.7984621537858504
NDCG для метода 4: 0.9583486499278216
NDCG для метода 5: 0.9204693415128716
NDCG для метода 6: 0.8626417015176465


Есть множество интересных рекомендаций, особенно понравились предсказания второй модели. Совсем плохих рекомендаций, чтобы мне не понравилось описание в IMBD не так много процентов 15%. При этом есть фильмов 10, которые я смотрел и очень люблю.

### Часть 5

### Сравнение рекомендаций и анализ полученных результатов

In [201]:
def compare_recs(recs_a, recs_b):
    movies_a = set([movie_id for _, movie_id in recs_a])
    movies_b = set([movie_id for _, movie_id in recs_b])

    common_movies = movies_a.intersection(movies_b)
    only_in_a = movies_a.difference(movies_b)
    only_in_b = movies_b.difference(movies_a)

    return common_movies, only_in_a, only_in_b

from scipy.stats import spearmanr

all_recs = [top1, top2, top3, top4, top5, top6]

for i in range(len(all_recs)):
    for j in range(i+1, len(all_recs)):
        common_movies, only_in_i, only_in_j = compare_recs(all_recs[i], all_recs[j])
        ranks_1 = np.argsort([rating for rating, _ in all_recs[i]])[::-1]
        ranks_2 = np.argsort([rating for rating, _ in all_recs[j]])[::-1]
        correlation, p_value = spearmanr(ranks_1, ranks_2)
        print(f"Сравнение модели {i+1} и модели {j+1}:")
        print(f"Количество общих фильмов: {len(common_movies)}")
        print(f"Correlation: {correlation}")

Сравнение модели 1 и модели 2:
Количество общих фильмов: 6
Correlation: 0.9272727272727272
Сравнение модели 1 и модели 3:
Количество общих фильмов: 6
Correlation: 0.6242424242424242
Сравнение модели 1 и модели 4:
Количество общих фильмов: 2
Correlation: 0.9515151515151514
Сравнение модели 1 и модели 5:
Количество общих фильмов: 2
Correlation: 0.9515151515151514
Сравнение модели 1 и модели 6:
Количество общих фильмов: 1
Correlation: 0.9515151515151514
Сравнение модели 2 и модели 3:
Количество общих фильмов: 5
Correlation: 0.6969696969696969
Сравнение модели 2 и модели 4:
Количество общих фильмов: 1
Correlation: 0.8787878787878788
Сравнение модели 2 и модели 5:
Количество общих фильмов: 1
Correlation: 0.8787878787878788
Сравнение модели 2 и модели 6:
Количество общих фильмов: 1
Correlation: 0.8787878787878788
Сравнение модели 3 и модели 4:
Количество общих фильмов: 4
Correlation: 0.5757575757575757
Сравнение модели 3 и модели 5:
Количество общих фильмов: 4
Correlation: 0.5757575757575757

### Анализ полученных результатов для метрик схожести
1) Коэффициент Жаккара удобен когда нужно учитывать только посмотрел пользователь фильм или нет, в данной ситуации не так хорош, так как не учитывает оценки для фильмов. Поэтому рекомендация 1 показала почти самый худший результат.
2) Скалярное произведение общих рейтингов уже напрямую учитывает оценки общих фильмов, но никак не учитывает фильмы которые не находятся в пересечении. Показал лучший результат (2 рекомендация) среди Простое усреднение по ближайшим соседям. Так же может быть чувствительной к экстремально высоким оценкам у небольшого числа элементов.
3) Скорректированная Корреляция Пирсона учитывает как сходство в оценках, так и различие от средних рейтингов, делая метрику более устойчивой к различиям в шкалах оценки у разных пользователей. При этом показал не самые лучшие результаты.

### Анализ полученных результатов для схем колаборативной фильтрации
1) Простое усреднение по ближайшим соседям прост в реализации, но не учитывает перекосы пользователя в оценках, что один может ставить в среднем всегда высокие или наоборот. Так же данный метод не учитывает личные склонности пользователя ставить более высокие или более низкие оценки, при выставлении финального предсказания.
2) Усреднение с учётом коррекции среднего исправляет искажения в оценках, связанные с индивидуальной тенденцией пользователя ставить более высокие или более низкие оценки, путём внесения корректировки на основе общего среднего рейтинга пользователя. Требует больше вычислений. Показал в среднем лучшие результаты чем 1 метод.